In [9]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

In [2]:
import tensorflow as tf

def identity_block(x, filters, strides=1):
    x_in = x

    x = tf.keras.layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='valid')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(filters*4, kernel_size=(1, 1), strides=strides, padding='valid')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)

    x = tf.keras.layers.Add()([x, x_in])
    x = tf.keras.layers.Activation('relu')(x)

    return x


def convolutional_block(x, filters,strides=1):
    x_in = x

    x = tf.keras.layers.Conv2D(filters, (1, 1), strides=2)(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)

    x = tf.keras.layers.Conv2D(filters*4, kernel_size=(1, 1), strides=strides, padding='valid')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)

    x_in = tf.keras.layers.Conv2D(filters*4, kernel_size=(1, 1), strides=2, padding='valid')(x_in)
    x_in = tf.keras.layers.BatchNormalization(axis=3)(x_in)

    x = tf.keras.layers.Add()([x, x_in])
    x = tf.keras.layers.Activation('relu')(x)
    return x


def ResNet50(input_shape=(64, 64, 3), classes=3):
    input_layer = tf.keras.Input(input_shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(input_layer)

    x = tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = convolutional_block(x, 64)
    x = identity_block(x,64)
    x = identity_block(x,64)

    x = convolutional_block(x, 128)
    for i in range(3):
        x = identity_block(x, 128)

    x = convolutional_block(x, 256)
    for i in range(5):
        x = identity_block(x, 256)

    x = convolutional_block(x,512)
    x = identity_block(x, 512)
    x = identity_block(x,512)


    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    output_layer = tf.keras.layers.Dense(classes, activation='softmax')(x)

    model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer, name='ResNet50')

    return model

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train[..., np.newaxis]/255.0, x_test[..., np.newaxis]/255.0

# one hot encode target values
trainY = tf.keras.utils.to_categorical(y_train)
testY = tf.keras.utils.to_categorical(y_test)

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

11501568/11490434 [==============================] - 0s 0us/step
(60000, 28, 28, 1) (10000, 28, 28, 1) (60000,) (10000,)


In [4]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.1,0.2],
    shear_range=0.2,
    zoom_range=0.2)

In [5]:
data_gen.fit(x_train)

In [6]:
mnist_model = ResNet50((28,28,1),classes=10)
mnist_model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 34, 34, 1)   0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 14, 14, 64)   3200        ['zero_padding2d[0][0]']         
                                                                                                  
 batch_normalization (BatchNorm  (None, 14, 14, 64)  256         ['conv2d[0][0]']                 
 alization)                                                                                

In [7]:
mnist_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [8]:
mnist_model.fit(x_train,y_train,epochs=10,steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 42s 205ms/step - loss: 0.6834 - accuracy: 0.8070
Epoch 2/10
100/100 [==============================] - 20s 204ms/step - loss: 0.1255 - accuracy: 0.9606
Epoch 3/10
100/100 [==============================] - 20s 203ms/step - loss: 0.0729 - accuracy: 0.9765
Epoch 4/10
100/100 [==============================] - 20s 203ms/step - loss: 0.0478 - accuracy: 0.9844
Epoch 5/10
100/100 [==============================] - 20s 203ms/step - loss: 0.0396 - accuracy: 0.9870
Epoch 6/10
100/100 [==============================] - 20s 203ms/step - loss: 0.0288 - accuracy: 0.9901
Epoch 7/10
100/100 [==============================] - 20s 203ms/step - loss: 0.0275 - accuracy: 0.9911
Epoch 8/10
100/100 [==============================] - 20s 203ms/step - loss: 0.0282 - accuracy: 0.9907
Epoch 9/10
100/100 [==============================] - 20s 203ms/step - loss: 0.0217 - accuracy: 0.9928
Epoch 10/10
100/100 [==============================] - 20s 203ms/step - l

In [10]:
mnist_model.evaluate(x=x_test, y=y_test)

313/313 [==============================] - 10s 27ms/step - loss: 0.1366 - accuracy: 0.9706


[0.13661859929561615, 0.9706000089645386]